In [1]:
import mlflow
from mlflow.models import infer_signature

import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
# Load the Iris dataset
X, y = datasets.load_iris(return_X_y=True)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Define the model hyperparameters
params = {
    "solver": "lbfgs",
    "max_iter": 1000,
    "multi_class": "auto",
    "random_state": 8888,
}

# Train the model
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

# Predict on the test set
y_pred = lr.predict(X_test)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)

In [7]:
mlflow.set_tracking_uri(uri="http://127.0.0.1:51973")
mlflow.set_experiment("MLflow Quickstart")

MlflowException: API request to endpoint /api/2.0/mlflow/experiments/get-by-name failed with error code 403 != 200. Response body: '<html><head> <title>ERROR: The requested URL could not be retrieved</title></head><body> <div id="titles"> <h1>ERROR</h1> <h2>The requested URL could not be retrieved</h2> </div> <hr>  <div id="content"> <p>The following error was encountered while trying to retrieve the URL: <a href="http://127.0.0.1:51973/api/2.0/mlflow/experiments/get-by-name?">http://127.0.0.1:51973/api/2.0/mlflow/experiments/get-by-name?</a></p>  <blockquote id="error"> <p><b>Access Denied.</b></p> </blockquote> <p>Following may the possible reasons</p> <p><b> 1) User not allowed to access internet from this IP</b></p> <p><b> 2) Requested URL is from a filtered domain</b></p>  <br> </div>  <hr></body></html> 
'

In [4]:
# Set our tracking server uri for logging
mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")

# Create a new MLflow Experiment
mlflow.set_experiment("MLflow Quickstart")

# Start an MLflow run
with mlflow.start_run():
    # Log the hyperparameters
    mlflow.log_params(params)

    # Log the loss metric
    mlflow.log_metric("accuracy", accuracy)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for iris data")

    # Infer the model signature
    signature = infer_signature(X_train, lr.predict(X_train))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train,
        registered_model_name="tracking-quickstart",
    )


MlflowException: API request to endpoint /api/2.0/mlflow/experiments/get-by-name failed with error code 403 != 200. Response body: '<html><head> <title>ERROR: The requested URL could not be retrieved</title></head><body> <div id="titles"> <h1>ERROR</h1> <h2>The requested URL could not be retrieved</h2> </div> <hr>  <div id="content"> <p>The following error was encountered while trying to retrieve the URL: <a href="http://127.0.0.1:8080/api/2.0/mlflow/experiments/get-by-name?">http://127.0.0.1:8080/api/2.0/mlflow/experiments/get-by-name?</a></p>  <blockquote id="error"> <p><b>Access Denied.</b></p> </blockquote> <p>Following may the possible reasons</p> <p><b> 1) User not allowed to access internet from this IP</b></p> <p><b> 2) Requested URL is from a filtered domain</b></p>  <br> </div>  <hr></body></html> 
'

In [ ]:
# Load the model back for predictions as a generic Python Function model
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

predictions = loaded_model.predict(X_test)

iris_feature_names = datasets.load_iris().feature_names

result = pd.DataFrame(X_test, columns=iris_feature_names)
result["actual_class"] = y_test
result["predicted_class"] = predictions

result[:4]
